In [ ]:
import json
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib import pyplot as plt

import add_path
import toolkit.allen_helpers.stimuli as st
import toolkit.pipeline.signal as ps
from toolkit.pipeline.data_io import SessionDirectory, FILES
from toolkit.pipeline.global_settings import GLOBAL_SETTINGS

pd.set_option('display.max_columns', None)

with open('config.json') as f:
    config = json.load(f)

## Get session and load data

In [2]:
session_id = config['session_id']
ecephys_structure_acronym = config['ecephys_structure_acronym']
print(f"Session ID: {session_id}")

Session ID: 766640955


In [ ]:
session_dir = SessionDirectory(session_id, ecephys_structure_acronym, cache_lfp=True)

probe_info = session_dir.load_probe_info()
if not session_dir.has_lfp_data:
    raise ValueError(f"Session {session_id} has no LFP data")

session = session_dir.session

preferred_orientation = session_dir.load_preferred_orientations().sel(
    layer=[GLOBAL_SETTINGS['layer_of_interest']]).values

layer_of_interest = GLOBAL_SETTINGS['layer_of_interest']

c:\Users\chen8\.conda\envs\allen\lib\site-packages\hdmf\spec\namespace.py:590: UserWarning: Ignoring the following cached namespace(s) because another version is already loaded:
core - cached version: 2.2.2, loaded version: 2.7.0
The loaded extension(s) may not be compatible with the cached extension(s) in the file. Please check the extension documentation and ignore this warning if these versions are compatible.
  self.warn_for_ignored_namespaces(ignored_namespaces)


In [4]:
stimulus_presentations = session.stimulus_presentations
session_type = session.session_type

stimulus_names = st.STIMULUS_NAMES[session_type]
drifting_gratings_stimuli = st.STIMULUS_CATEGORIES[session_type]['drifting_gratings']
natural_movies_stimuli = st.STIMULUS_CATEGORIES[session_type]['natural_movies']

## Analyze data

### Load LFP power

In [ ]:
# # Filter conditions 
# filter_orientation = False  # whether use only preferred orientation
# valid_conditions = ps.filter_conditions(conditions[0])
# if filter_orientation:
#     valid_conditions = valid_conditions.sel(orientation=preferred_orientation)
# valid_presentations = np.sort(np.concatenate([conditions[1][c] for c in valid_conditions.values.ravel()]))
# aligned_lfp = aligned_lfp.sel(presentation_id=valid_presentations)
# valid_trials = st.choose_trials(stimulus_trials, valid_presentations)